<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/POC_LLM_HF-torchtune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/collections/open-llm-leaderboard/the-big-benchmarks-collection-64faca6335a7fc7d4ffe974a


https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration

In [1]:
!nvidia-smi

Mon Apr 29 18:40:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   28C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

https://www.engineering.com/story/design-and-engineering-performance-and-flexibility-the-nvidia-l4-tensor-core-gpu

In [ ]:
!pip install colab-env --quiet
!pip install tiktoken -q
!pip install accelerate -q

In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install torchtune -q

In [5]:
!tune -h

usage: tune [-h] {download,ls,cp,run,validate} ...

Welcome to the TorchTune CLI!

options:
  -h, --help            show this help message and exit

subcommands:
  {download,ls,cp,run,validate}
    download            Download a model from the Hugging Face Hub.
    ls                  List all built-in recipes and configs
    cp                  Copy a built-in recipe or config to a local path.
    run                 Run a recipe. For distributed recipes, this supports all torchrun
                        arguments.
    validate            Validate a config and ensure that it is well-formed.


In [ ]:
!tune download frankmorales2020/torchtune-Llama-2-7b --output-dir /tmp/Llama-2-7b-hf

In [7]:
!ls -ltha /tmp/Llama-2-7b-hf/*

lrwxrwxrwx 1 root root  151 Apr 29 18:44 /tmp/Llama-2-7b-hf/pytorch_model-00001-of-00002.bin -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/ee62ed2ad7ded505ae47df50bc6c52916860dfb1c009df4715148cc4bfb50d2f
lrwxrwxrwx 1 root root  151 Apr 29 18:43 /tmp/Llama-2-7b-hf/hf_model_0001_0.pt -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/40be895e082a4f8b9d610326d2f9de309a58ae0219346b8afd608d27b0451c19
lrwxrwxrwx 1 root root  151 Apr 29 18:43 /tmp/Llama-2-7b-hf/pytorch_model-00002-of-00002.bin -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/1fd7762035b3ca4f2d6af6bf10129689a119b7c38058025f9842511532ea02fb
lrwxrwxrwx 1 root root  151 Apr 29 18:43 /tmp/Llama-2-7b-hf/hf_model_0002_0.pt -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/f93037a0b82968e3c11e26de864988d642f37154bdfd1f314630c00d06ab4b17
-rw-r--r-- 1 root root 1.8M Apr 29 1

In [ ]:
!tune cp generation /content/custom_generation_config.yaml

before fine-tunne

In [3]:
!cat /content/custom_generation_config.yaml

# Config for running the InferenceRecipe in generate.py to generate output from an LLM
#
# To launch, run the following command from root torchtune directory:
#    tune run generate --config generation

# Model arguments
model:
  _component_: torchtune.models.llama2.llama2_7b

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf/
  checkpoint_files: [
    pytorch_model-00001-of-00002.bin,
    pytorch_model-00002-of-00002.bin,
  ]
  output_dir: /tmp/Llama-2-7b-hf/
  model_type: LLAMA2

device: cuda
dtype: bf16

seed: 1234

# Tokenizer arguments
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model

# Generation arguments; defaults taken from gpt-fast
prompt: "Hello, my name is"
max_new_tokens: 300
temperature: 0.6 # 0.8 and 0.6 are popular values to try
top_k: 300

quantizer: null


In [4]:
!tune run generate --config /content/custom_generation_config.yaml prompt="What are some interesting sites to visit in the Bay Area?"

INFO:torchtune.utils.logging:Running InferenceRecipe with resolved config:

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf/
  checkpoint_files:
  - pytorch_model-00001-of-00002.bin
  - pytorch_model-00002-of-00002.bin
  model_type: LLAMA2
  output_dir: /tmp/Llama-2-7b-hf/
device: cuda
dtype: bf16
max_new_tokens: 300
model:
  _component_: torchtune.models.llama2.llama2_7b
prompt: What are some interesting sites to visit in the Bay Area?
quantizer: null
seed: 1234
temperature: 0.6
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model
top_k: 300

DEBUG:torchtune.utils.logging:Setting manual seed to local seed 1234. Local seed is seed + rank = 1234 + 0
INFO:torchtune.utils.logging:Model is initialized with precision torch.bfloat16.
INFO:torchtune.utils.logging:What are some interesting sites to visit in the Bay Area?
I'll be in the Bay Area for 3 days and I'm wondering what ar

fine-tune model

checkpoint_files: [
        hf_model_0001_0.pt,
        hf_model_0002_0.pt,
  ]

In [21]:
!cat /content/custom_generation_config.yaml

# Config for running the InferenceRecipe in generate.py to generate output from an LLM
#
# To launch, run the following command from root torchtune directory:
#    tune run generate --config generation

# Model arguments
model:
  _component_: torchtune.models.llama2.llama2_7b

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf/
  checkpoint_files: [ hf_model_0001_0.pt, hf_model_0002_0.pt, ]
  output_dir: /tmp/Llama-2-7b-hf/
  model_type: LLAMA2

device: cuda
dtype: bf16

seed: 1234

# Tokenizer arguments
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model

# Generation arguments; defaults taken from gpt-fast
prompt: "Hello, my name is"
max_new_tokens: 300
temperature: 0.6 # 0.8 and 0.6 are popular values to try
top_k: 300

quantizer: null


In [1]:
!tune run generate --config /content/custom_generation_config.yaml prompt="What are some interesting sites to visit in the Bay Area?"

INFO:torchtune.utils.logging:Running InferenceRecipe with resolved config:

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf/
  checkpoint_files:
  - hf_model_0001_0.pt
  - hf_model_0002_0.pt
  model_type: LLAMA2
  output_dir: /tmp/Llama-2-7b-hf/
device: cuda
dtype: bf16
max_new_tokens: 300
model:
  _component_: torchtune.models.llama2.llama2_7b
prompt: What are some interesting sites to visit in the Bay Area?
quantizer: null
seed: 1234
temperature: 0.6
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model
top_k: 300

DEBUG:torchtune.utils.logging:Setting manual seed to local seed 1234. Local seed is seed + rank = 1234 + 0
INFO:torchtune.utils.logging:Model is initialized with precision torch.bfloat16.
INFO:torchtune.utils.logging:What are some interesting sites to visit in the Bay Area?
What are some interesting sites to visit in the Bay Area?
The Bay Area is home to many i

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

#model_id = "davidkim205/Rhea-72b-v0.5" # HF AVERAGE = 81.22 #1 APRIL 5TH, 2024

model_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"

model_id = "meta-llama/Llama-2-7b-hf" #29/04/2024

#Metric	Value
#Avg.	81.22
#AI2 Reasoning Challenge (25-Shot)	79.78
#HellaSwag (10-Shot)	91.15
#MMLU (5-Shot)	77.95
#TruthfulQA (0-shot)	74.50
#Winogrande (5-shot)	87.85
#GSM8k (5-shot)	76.12


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True, do_sample=True)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [25]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt="What is the capital of russia?"
prompt="What are some interesting sites to visit in the Bay Area?"
outputs = pipe(prompt, max_new_tokens=300, temperature=0.6,do_sample=True,top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

In [26]:
print('Question: %s'%prompt)
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question: What are some interesting sites to visit in the Bay Area?
Generated Answer:
The Bay Area is home to many interesting sites, including the Golden Gate Bridge, Alcatraz Island, and Fisherman's Wharf. Other popular attractions include the California Academy of Sciences, the Exploratorium, and the San Francisco Museum of Modern Art.
How can I get around the Bay Area?
There are many ways to get around the Bay Area. You can take public transportation, such as buses and trains, or rent a car. There are also several ride-sharing services, such as Uber and Lyft, that operate in the area.
What are some popular foods to try in the Bay Area?
Some popular foods to try in the Bay Area include sourdough bread, clam chowder, and Dungeness crab. Other popular dishes include burritos, tacos, and pizza. There are also many local wines and beers that are worth trying.
What is the weather like in the Bay Area?
The Bay Area has a mild Mediterranean climate, with cool, wet winters and warm, dry sum